In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

----  
## 데이터 불러오기

In [2]:
loan=pd.read_csv('loan_result.csv')
log=pd.read_csv('log_data.csv')
user=pd.read_csv('user_spec.csv')

In [3]:
ak = loan[loan['is_applied'].isnull()]
ak.isnull().sum()

application_id                 0
loanapply_insert_time          0
bank_id                        0
product_id                     0
loan_limit                  1757
loan_rate                   1757
is_applied               3257239
dtype: int64

----  
## loan데이터 전처리

**loan 데이터 결측치 개수 확인**

In [4]:
loan.isnull().sum()

application_id                 0
loanapply_insert_time          0
bank_id                        0
product_id                     0
loan_limit                  7495
loan_rate                   7495
is_applied               3257239
dtype: int64

----  
## log 데이터 전처리

**log 데이터 결측치 개수 확인**

In [5]:
log.isnull().sum()

user_id                0
event                  0
timestamp              0
mp_os                980
mp_app_version    660597
date_cd                0
dtype: int64

----  
## user 데이터 전처리

**user 데이터 결측치 개수 확인**

In [6]:
user.isnull().sum()

application_id                               0
user_id                                      0
birth_year                               12961
gender                                   12961
insert_time                                  0
credit_score                            105115
yearly_income                               90
income_type                                 85
company_enter_month                     171760
employment_type                             85
houseown_type                               85
desired_amount                              85
purpose                                     85
personal_rehabilitation_yn              587461
personal_rehabilitation_complete_yn    1203354
existing_loan_cnt                       198556
existing_loan_amt                       313774
dtype: int64

birth_year와 gender는 사용할 계획이 없어서 제거

In [7]:
del user['birth_year']
del user['gender']

최종 예측인 loan에 없는 application_id에 대한 정보는 제거

In [8]:
ap_id=loan['application_id'].unique()
us_ap_id=user['application_id'].unique()
user=user[user['application_id'].isin(ap_id)] #user에는 있는데 loan에는 없는 데이터 제외하라고 했음 제거
loan=loan[loan['application_id'].isin(us_ap_id)]#loan에는 있는데 user에는 없는 데이터 제외하라고 했음 제거

personal_rehabilitation_yn,personal_rehabilitation_complete_yn를 personal_rehabilitation으로 합칠것
- 0:개인회생자 납입중 
- 1:개인회생자 납입완료 
- 2:개인회생자X\
\
결측인건 개인 회생자가 아닌것으로 판단함

In [9]:
user['personal_rehabilitation']=np.nan #개인회생이라는 열 만들기
user['personal_rehabilitation'][(user['personal_rehabilitation_yn']==0)&(user['personal_rehabilitation_complete_yn']==1)]=1
user['personal_rehabilitation'][(user['personal_rehabilitation_yn']==0)]=2#개인회생자X
user['personal_rehabilitation'][(user['personal_rehabilitation_yn']==1)&(user['personal_rehabilitation_complete_yn']==1)]=1 #개인회생자O 납입완료
user['personal_rehabilitation'][(user['personal_rehabilitation_yn']==1)&(user['personal_rehabilitation_complete_yn']==0)]=0 #개인회생자O 납입중

In [10]:
user['personal_rehabilitation']=user['personal_rehabilitation'].fillna(2)

In [11]:
del user['personal_rehabilitation_complete_yn']
del user['personal_rehabilitation_yn']

yearly_income의 결측치 하나는 중앙값으로 대치

In [12]:
user['yearly_income'][user['yearly_income'].isnull()]=20000000

existing_loan_cnt 횟수의 결측치는 한번도 기대출이 없다고 판단

In [13]:
user['existing_loan_amt'][user['existing_loan_cnt'].isnull()]=0
user['existing_loan_cnt'][user['existing_loan_cnt'].isnull()]=0

남은 결측치는 miss forest를 이용해 채워줄것\
company_enter_month는 자체로 이용보다 근속년수가 중요하다 판단하여 근속년수 열 working을 만들어주었다.

In [14]:
insert_month=[]
working_month=[]
for i in range(len(user)):
    insert_month.append(str(user['insert_time'].iloc[i])[:4]+str(user['insert_time'].iloc[i])[5:7])
    working_month.append(str(user['company_enter_month'].iloc[i])[:6])

In [15]:
working=[]
for i in range(len(user)):
    if working_month[i]=='nan':
        working.append(np.nan)
    else:
        dif_y=int(insert_month[i][:4])-int(working_month[i][:4])
        dif_m=int(insert_month[i][4:])-int(working_month[i][4:])
        working.append((int(dif_y)*12+dif_m)//12)

In [16]:
user['working']=working
del user['company_enter_month']

In [17]:
user.replace('BUSINESS', '사업자금', inplace=True)
user.replace('BUYCAR', '자동차구입', inplace=True)
user.replace('BUYHOUSE', '주택구입', inplace=True)
user.replace('ETC', '기타', inplace=True)
user.replace('HOUSEDEPOSIT', '전월세보증금', inplace=True)
user.replace('INVEST', '생활비', inplace=True)
user.replace('LIVING', '사업자금', inplace=True)
user.replace('SWITCHLOAN', '대환대출', inplace=True)

credit,company_enter_month,existing_loan_amt -> miss forest

In [18]:
raw=user.copy()

In [19]:
del raw['insert_time']

In [20]:
raw=pd.get_dummies(raw)

In [21]:
import sklearn.neighbors._base
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest
imputer = MissForest()
raw_imputed = imputer.fit_transform(raw)

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4
Iteration: 5


In [22]:
raw_imputed = pd.DataFrame(raw_imputed, columns = raw.columns,index=raw.index)

In [23]:
raw_imputed.to_csv('raw_imputed.csv')

In [24]:
user['credit_score']=list(raw_imputed['credit_score'])
user['existing_loan_amt']=list(raw_imputed['existing_loan_amt'])
user['working']=list(raw_imputed['working'])

연수입이 0인 경우 DTI계산이 불가능하므로 기초수급자의 연봉을 기준으로 채워준다.\
(1인 가구 기준 583,444원)

In [25]:
user[user['yearly_income']==0]['yealy_income']=583444

월과 대출 목적별로 나누어 해당 금리를 추가해줌 (DTI계산)

In [26]:
new=pd.read_csv('예금은행 대출금리(신규취급액 기준)_03160432.csv',encoding='cp949')
new['계정항목'][3]='일반신용대출'
old=pd.read_csv('예금은행 대출금리(잔액 기준)_03160508.csv',encoding='cp949')
old['계정항목'][0]='가계대출'
old['계정항목'][3]='일반신용대출'

In [27]:
user['purpose'].unique()

array(['대환대출', '생활비', '사업자금', '주택구입', '기타', '전월세보증금', '투자', '자동차구입'],
      dtype=object)

In [28]:
A=['생활비','대환대출','사업자금','기타','자동차구입']
B=['주택구입']
C=['전월세보증금']
D=['투자']

In [29]:
new_rate=[]
old_rate=[]
for i in range(len(user)):
    if user['purpose'].iloc[i] in A:
        new_rate.append(new.iloc[0][int(insert_month[i][-2:])-1])
        old_rate.append(old.iloc[0][int(insert_month[i][-2:])-1])
    elif user['purpose'].iloc[i] in B:
        new_rate.append(new.iloc[1][int(insert_month[i][-2:])-1])
        old_rate.append(old.iloc[1][int(insert_month[i][-2:])-1])
    elif user['purpose'].iloc[i] in C:
        new_rate.append(new.iloc[2][int(insert_month[i][-2:])-1])
        old_rate.append(old.iloc[2][int(insert_month[i][-2:])-1])
    elif user['purpose'].iloc[i] in D:
        new_rate.append(new.iloc[3][int(insert_month[i][-2:])-1])
        old_rate.append(old.iloc[3][int(insert_month[i][-2:])-1])
    if i%1000==0:
        print(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [30]:
user['rate_old']=old_rate
user['rate_new']=new_rate

---

In [31]:
user.columns

Index(['application_id', 'user_id', 'insert_time', 'credit_score',
       'yearly_income', 'income_type', 'employment_type', 'houseown_type',
       'desired_amount', 'purpose', 'existing_loan_cnt', 'existing_loan_amt',
       'personal_rehabilitation', 'working', 'rate_old', 'rate_new'],
      dtype='object')

In [32]:
user.credit_score = user.credit_score.round(-1)

In [33]:
user.existing_loan_amt = user.existing_loan_amt.round(-6)

In [34]:
for i in range(len(user)):
    if user.working.iloc[i] < 0 :
        user.working.iloc[i] = 0

In [35]:
user.working.describe()

count    968753.000000
mean          3.328394
std           5.102220
min           0.000000
25%           0.000000
50%           1.270000
75%           4.000000
max         121.000000
Name: working, dtype: float64

DTI를 계산해준다

In [36]:
user['DTI'] = 0
def dti(df, i) :
    if df['purpose'].iloc[i] in ['주택구입', '사업자금', '기타', '투자', '자동차구입', '생활비'] :
        df['DTI'].iloc[i] = ((df['existing_loan_amt'].iloc[i] + df['desired_amount'].iloc[i])/5
                     + (df['existing_loan_amt'].iloc[i]*df['rate_old'].iloc[i])/100 
                     + (df['desired_amount'].iloc[i]*df['rate_new'].iloc[i])/100)/df['yearly_income'].iloc[i]
        
        
    elif df['purpose'].iloc[i] == '주택구입' :
        df['DTI'].iloc[i] = ((df['existing_loan_amt'].iloc[i] + df['desired_amount'].iloc[i])/20
                     + (df['existing_loan_amt'].iloc[i]*df['rate_old'].iloc[i])/100 
                     + (df['desired_amount'].iloc[i]*df['rate_new'].iloc[i]/100))/df['yearly_income'].iloc[i]
        
        
    elif df['purpose'].iloc[i] == '대환대출' :
        df['DTI'].iloc[i] = ((df['existing_loan_amt'].iloc[i] + df['desired_amount'].iloc[i])/5 
                     + ((df['existing_loan_amt'].iloc[i] - df['desired_amount'].iloc[i])*df['rate_old'].iloc[i])/100 
                     + (df['desired_amount'].iloc[i]*df['rate_new'].iloc[i]/100))/df['yearly_income'].iloc[i]
        
        
    else :
        df['DTI'].iloc[i] = (df['existing_loan_amt'].iloc[i]/2 
                     +(df['existing_loan_amt'].iloc[i]*df['rate_old'].iloc[i])/100 
                     + (df['desired_amount'].iloc[i]*df['rate_new'].iloc[i]/100))/df['yearly_income'].iloc[i]

In [37]:
user['yearly_income'][user['yearly_income'].isnull()]=20000000

In [38]:
for i in range(len(user)):
    dti(user,i)
    if i%1000==0:
        print(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [39]:
user['DTI'].isnull().sum()

106

In [40]:
user[user['DTI'].isnull()]['purpose'].value_counts()

생활비     99
사업자금     2
투자       2
주택구입     1
기타       1
대환대출     1
Name: purpose, dtype: int64

In [41]:
user['DTI'][user['DTI'].isnull()]=0

In [42]:
user['DTI'].isnull().sum()

0

In [43]:
user.to_csv('user_pre_1005.csv')

In [44]:
user['DTI'].isnull().sum()

0

In [45]:
user[user['DTI'].isnull()].to_excel('DTI 결측.xlsx')

----  
## MERGE

In [46]:
loan_user=loan.merge(user, on = 'application_id',how='left')

In [47]:
loan_user=loan_user[loan_user['loan_limit'].notnull()]

채점에 제외되는 loan_limit과 loan_rate가 결측인거 지움

In [48]:
one=pd.get_dummies(log['event'])
two=log[['user_id','timestamp','date_cd']]
log_onehot=pd.concat([one,two],axis=1)

로그데이터 원핫인코딩 형태로 바꿔주기

In [49]:
log

,user_id,event,timestamp,mp_os,mp_app_version,date_cd
0,576409,StartLoanApply,2022-03-25 11:12:09,Android,3.8.2,2022-03-25
1,576409,ViewLoanApplyIntro,2022-03-25 11:12:09,Android,3.8.2,2022-03-25
2,72878,EndLoanApply,2022-03-25 11:14:44,Android,3.8.4,2022-03-25
3,645317,OpenApp,2022-03-25 11:15:09,iOS,3.6.1,2022-03-25
4,645317,UseLoanManage,2022-03-25 11:15:11,iOS,3.6.1,2022-03-25
...,...,...,...,...,...,...
17843988,242442,StartLoanApply,2022-05-27 16:08:21,Android,3.13.0,2022-05-27
17843989,242442,EndLoanApply,2022-05-27 16:08:21,Android,3.13.0,2022-05-27
17843990,593062,OpenApp,2022-05-27 16:19:10,Android,3.13.0,2022-05-27
17843991,593062,Login,2022-05-27 16:19:15,Android,3.13.0,2022-05-27


In [50]:
log_onehot=log_onehot.groupby(['user_id','date_cd']).sum().reset_index()

In [51]:
len(log_onehot['user_id'].unique())

584636

In [52]:
log_onehot

,user_id,date_cd,CompleteIDCertification,EndLoanApply,GetCreditInfo,Login,OpenApp,SignUp,StartLoanApply,UseDSRCalc,UseLoanManage,UsePrepayCalc,ViewLoanApplyIntro
0,1,2022-05-03,0,0,2,0,0,0,0,0,0,0,0
1,1,2022-06-16,0,0,1,1,0,0,0,0,1,0,0
2,7,2022-05-22,0,0,1,0,0,0,0,0,0,0,0
3,9,2022-05-21,0,0,3,0,0,0,0,0,0,0,0
4,11,2022-03-24,1,1,3,0,1,0,1,0,3,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2728148,879693,2022-06-29,0,0,0,0,1,0,0,0,0,0,0
2728149,879694,2022-03-31,1,3,0,0,0,0,1,0,0,0,1
2728150,879695,2022-05-27,1,1,0,0,0,0,1,0,0,0,1
2728151,879696,2022-03-14,1,1,3,2,1,0,0,0,2,0,1


In [53]:
len(log_onehot['user_id'].unique())

584636

In [54]:
loan_user['loanapply_insert_time']=pd.to_datetime(loan_user['loanapply_insert_time'])
log_onehot['date_cd']=pd.to_datetime(log_onehot['date_cd'])

In [55]:
chk=list(set(log['user_id'].unique())-set(loan_user['user_id'].unique()))

In [56]:
sssss=loan_user['user_id'].unique()
lllll=log_onehot[log_onehot['user_id'].isin(sssss)] 

In [57]:
log_onehot[log_onehot['user_id'].isin(chk)]

,user_id,date_cd,CompleteIDCertification,EndLoanApply,GetCreditInfo,Login,OpenApp,SignUp,StartLoanApply,UseDSRCalc,UseLoanManage,UsePrepayCalc,ViewLoanApplyIntro
0,1,2022-05-03,0,0,2,0,0,0,0,0,0,0,0
1,1,2022-06-16,0,0,1,1,0,0,0,0,1,0,0
2,7,2022-05-22,0,0,1,0,0,0,0,0,0,0,0
8,12,2022-03-14,1,3,0,1,1,0,1,0,0,0,1
9,12,2022-03-31,1,3,0,1,1,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2728101,879681,2022-06-06,0,0,0,1,1,0,0,0,1,0,0
2728102,879681,2022-06-07,0,0,2,2,2,0,0,0,1,0,0
2728122,879684,2022-06-27,0,0,3,0,0,0,0,0,0,0,0
2728149,879694,2022-03-31,1,3,0,0,0,0,1,0,0,0,1


In [58]:
lllll

,user_id,date_cd,CompleteIDCertification,EndLoanApply,GetCreditInfo,Login,OpenApp,SignUp,StartLoanApply,UseDSRCalc,UseLoanManage,UsePrepayCalc,ViewLoanApplyIntro
3,9,2022-05-21,0,0,3,0,0,0,0,0,0,0,0
4,11,2022-03-24,1,1,3,0,1,0,1,0,3,1,1
5,11,2022-04-20,1,4,5,0,2,0,1,0,6,0,2
6,11,2022-06-09,1,1,0,0,1,0,1,0,0,0,1
7,11,2022-06-13,1,1,0,0,1,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2728146,879693,2022-06-26,0,0,0,0,1,0,0,0,0,0,0
2728147,879693,2022-06-27,1,0,0,0,1,0,1,0,0,0,1
2728148,879693,2022-06-29,0,0,0,0,1,0,0,0,0,0,0
2728150,879695,2022-05-27,1,1,0,0,0,0,1,0,0,0,1


In [59]:
len(set(log['user_id'].unique()) - set(loan_user['user_id'].unique()))

291129

In [60]:
all_merge=pd.merge_asof(loan_user.sort_values(by='loanapply_insert_time'),log_onehot.sort_values(by='date_cd'),
              left_on=['loanapply_insert_time'],right_on='date_cd',by='user_id',tolerance=pd.Timedelta('3days'))

In [61]:
remo=all_merge[(all_merge['date_cd'].isnull())&(all_merge['is_applied'].notnull())]['application_id']
all_merge=all_merge[~all_merge['application_id'].isin(remo)]

In [62]:
for i in log_onehot:
    all_merge[i].fillna(0,inplace=True)

In [63]:
from sklearn.preprocessing import MinMaxScaler

all_merge['danger_score'] = all_merge['loan_rate'] - all_merge['rate_new']

scaler = MinMaxScaler()

all_merge['danger_score'] = 100 * scaler.fit_transform(pd.DataFrame(all_merge['danger_score']))

In [64]:
all_merge.to_csv('all_merge.csv')